# Método Over 2.5 FT

**Critérios do Método**

a) Calculamos a Porcentagem de Over 2.5 FT

b) Calculamos a Porcentagem de BTTS

c) Calculamos a Média de Gols Feitos e Sofridos pelo Mandante

d) Calculamos a Média de Gols Feitos e Sofridos pelo Visitante

e) Calculamos o Desvio Padrão e o Coeficiente de Variação dessas médias

f) Definimos nosso Filtro Estatístico:
1. Porcentagem de Over 2.5 FT Home > 50% 
2. Porcentagem de Over 2.5 FT Away > 50%
2. Porcentagem de BTTS Home > 50%
2. Porcentagem de BTTS Away > 50%
3. Média de Gols Feitos Home > 1 
4. Média de Gols Sofridos Home > 1
5. Média de Gols Feitos Away > 1
6. Média de Gols Sofridos Away > 1
7. CV de Gols Feitos Home < 1 
8. CV de Gols Sofridos Home < 1
9. CV de Gols Feitos Away < 1
10. CV de Gols Sofridos Away < 1

g) Todos os jogos que possuem esses dados entramos em Pré-Live com a Odd oferecida pela casa.



# Importando as Bibliotecas e o DataFrame 

In [ ]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# df = pd.read_csv("https://www.football-data.co.uk/mmz4281/2122/E0.csv")
# df['Liga'] = 'Inglaterra 1'

# df = df[['Liga','Date','HomeTeam','AwayTeam','HTHG','HTAG','HTR','FTHG','FTAG','FTR','B365H','B365D','B365A','B365>2.5']]
# df.columns = ['League','Date','Home','Away','Goals_H_HT','Goals_A_HT','Result_HT','Goals_H_FT','Goals_A_FT','Result_FT','Odds_H','Odds_D','Odds_A','Odds_Over25_FT']

In [ ]:
# DataFrame 2021/2022
df = pd.read_excel("https://github.com/futpythontrader/YouTube/blob/main/DataFrame_2021_2022.xlsx?raw=true")

# DataFrame 2020/2021
# df = pd.read_excel("https://github.com/futpythontrader/YouTube/blob/main/DataFrame_2020_2021.xlsx?raw=true")

# Porcentagem de Gols

In [ ]:
df['Total_Goals_FT'] = df.apply(lambda row: (row['Goals_H_FT'] + row['Goals_A_FT']), axis=1) 

df['Over25_FT'] = df.apply(lambda row: 1 if (row['Total_Goals_FT'] > 2) else 0, axis=1) 
df['BTTS'] = df.apply(lambda row: 1 if (row['Goals_H_FT'] > 0 and row['Goals_A_FT'] > 0) else 0, axis=1) 

In [ ]:
df['Porc_Over25FT_Home'] = (df.groupby('Home')['Over25_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)
df['Porc_Over25FT_Away'] = (df.groupby('Away')['Over25_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)

df['Porc_BTTS_Home'] = (df.groupby('Home')['BTTS'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)
df['Porc_BTTS_Away'] = (df.groupby('Away')['BTTS'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True) * 100).shift(1)

# Médias, Desvio Padrão e Coeficiente de Variação

In [ ]:
df['Media_Gols_Feitos_Home'] = df.groupby('Home')['Goals_H_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Gols_Sofridos_Home'] = df.groupby('Home')['Goals_A_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

df['Media_Gols_Feitos_Away'] = df.groupby('Away')['Goals_A_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Gols_Sofridos_Away'] = df.groupby('Away')['Goals_H_FT'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
df['DP_Gols_Feitos_Home'] = df.groupby('Home')['Goals_H_FT'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
df['DP_Gols_Sofridos_Home'] = df.groupby('Home')['Goals_A_FT'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)

df['DP_Gols_Feitos_Away'] = df.groupby('Away')['Goals_A_FT'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
df['DP_Gols_Sofridos_Away'] = df.groupby('Away')['Goals_H_FT'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)

In [ ]:
df['CV_Gols_Feitos_Home'] = (df['DP_Gols_Feitos_Home'] / df['Media_Gols_Feitos_Home']).shift(1)
df['CV_Gols_Sofridos_Home'] = (df['DP_Gols_Sofridos_Home'] / df['Media_Gols_Sofridos_Home']).shift(1)

df['CV_Gols_Feitos_Away'] = (df['DP_Gols_Feitos_Away'] / df['Media_Gols_Feitos_Away']).shift(1)
df['CV_Gols_Sofridos_Away'] = (df['DP_Gols_Sofridos_Away'] / df['Media_Gols_Sofridos_Away']).shift(1)

In [ ]:
df0 = df.groupby(['Home']).get_group('Chelsea')

In [ ]:
df0 = df0[['Home','Away','Goals_H_FT','Goals_A_FT','Media_Gols_Feitos_Home','Media_Gols_Sofridos_Home','CV_Gols_Feitos_Home','CV_Gols_Sofridos_Home']]
df0

# Ajustando o DataFrame

In [ ]:
df1 = df[['Home','Away','Odds_H','Odds_Over25_FT','Goals_H_FT','Goals_A_FT','Total_Goals_FT','Porc_Over25FT_Home','Porc_Over25FT_Away','Porc_BTTS_Home','Porc_BTTS_Away','Media_Gols_Feitos_Home','Media_Gols_Sofridos_Home','Media_Gols_Feitos_Away','Media_Gols_Sofridos_Away','CV_Gols_Feitos_Home','CV_Gols_Sofridos_Home','CV_Gols_Feitos_Away','CV_Gols_Sofridos_Away']]
df1 = df1.dropna()

# Filtro Estatístico

In [ ]:
# Selecionando o Filtro Estatítisco
flt2 = (df1.Porc_Over25FT_Home > 50) & (df1.Porc_Over25FT_Away > 50) & (df1.Porc_BTTS_Home > 50) & (df1.Porc_BTTS_Away > 50) & (df1.Media_Gols_Feitos_Home > 1) & (df1.Media_Gols_Sofridos_Home > 1) & (df1.Media_Gols_Feitos_Away > 1) & (df1.Media_Gols_Sofridos_Away > 1) & (df1.CV_Gols_Feitos_Home < 1) & (df1.CV_Gols_Sofridos_Home < 1) & (df1.CV_Gols_Feitos_Away < 1) & (df1.CV_Gols_Sofridos_Away < 1)
df2 = df1[flt2];
# Ajustando o Índice
df2.reset_index(inplace=True, drop=True)
df2.index = df2.index.set_names(['Nº'])
df2 = df2.rename(index=lambda x: x + 1)

# Resultados

In [ ]:
Stake = 1
green = Stake * (df2.Odds_Over25_FT - 1) 
red = -Stake

# Over 2.5 FT
df2.loc[((df2['Total_Goals_FT']) > 2), 'Profit_Over25'] = green
df2.loc[((df2['Total_Goals_FT']) <= 2), 'Profit_Over25'] = red
df2['Profit_acu_Over25'] = df2.Profit_Over25.cumsum()
df2.Profit_acu_Over25.plot(title='Over 2.5 FT', xlabel='Quantidade de Apostas', ylabel='Stakes')

In [ ]:
df2['Profit_acu_Over25'].tail(1)